In [14]:
import os
import random
from google.cloud import vision

# Set up your credentials and initialize the Vision API client
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "photo-haiku.json"
print("Google Cloud Vision API is installed and ready!")
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

client = vision.ImageAnnotatorClient()
print("Vision API client successfully initialized!")

def analyze_image(image_path): #Analyzes an image for object labels using Google Cloud Vision API
    # Read image file
    with open(image_path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # Object detection
    label_response = client.label_detection(image=image)
    labels = label_response.label_annotations

    print("\n🎯 Detected Labels:")
    for label in labels:
        print(f"- {label.description} (Confidence: {label.score:.2f})")

    # Face emotion detection
    face_response = client.face_detection(image=image)
    faces = face_response.face_annotations
    if not faces:
        print("\n😊 No faces detected.")
    else:
        print("\n😊 Facial Emotion Analysis:")
        for i, face in enumerate(faces):
            print(f"\n👤 Face {i+1}:")
            print(f"   - Joy: {face.joy_likelihood}")
            print(f"   - Sorrow: {face.sorrow_likelihood}")
            print(f"   - Anger: {face.anger_likelihood}")
            print(f"   - Surprise: {face.surprise_likelihood}")

    # Return only the label descriptions
    return [label.description for label in labels]

def count_syllables(word): #heuristic syllable counter.
    vowels = "aeiouy"
    word = word.lower().strip()
    count = 0
    prev_was_vowel = False
    for letter in word:
        if letter in vowels:
            if not prev_was_vowel:
                count += 1
            prev_was_vowel = True
        else:
            prev_was_vowel = False
    if word.endswith("e"):
        count -= 1
    return max(1, count)

def choose_label_for(required, pool, fallback_dict):
    """
    Chooses a word from the pool that has exactly the required number of syllables.
    Removes the chosen word from the pool to avoid repetition.
    If no word is found, returns a fallback word.
    """
    candidates = [word for word in pool if count_syllables(word) == required]
    if candidates:
        chosen = random.choice(candidates)
        pool.remove(chosen)
        return chosen
    else:
        return fallback_dict.get(required, "day")  # default fallback

def create_template_haiku(labels):
    """Generates a haiku using pre-defined templates.
    Each template has a placeholder {a} that must be filled by a label word
    whose syllable count makes the line add up to 5 or 7 syllables.
    """
    # Make a copy of label words to use (so we don't repeat them)
    label_pool = labels[:]
    # Define fallback words for different syllable requirements
    fallbacks = {1: "day", 2: "sunrise", 3: "evening"}

    # Define templates for 5-syllable lines.
    # Each tuple is (template, required syllables for {a})
    line1_templates = [
       ("Soft {a} glows", 3),   # "Soft" (1) + "glows" (1) = 2; needs 3 more to make 5.
       ("Calm {a} sings", 3)    # "Calm" (1) + "sings" (1) = 2; needs 3 more.
    ]
    line3_templates = [
       ("Quiet {a} dreams", 2),  # "Quiet" (2) + "dreams" (1) = 3; needs 2 more.
       ("Faint {a} calls", 3)     # "Faint" (1) + "calls" (1) = 2; needs 3 more.
    ]
    # Define templates for 7-syllable lines.
    line2_templates = [
       ("Beneath the {a} sky", 3),         # "Beneath" (2) + "the" (1) + "sky" (1) = 4; needs 3.
       ("In the {a} light of night", 2),    # "In"(1)+ "the"(1)+ "light"(1)+ "of"(1)+ "night"(1)=5; needs 2.
       ("Under {a} stars, dreams awake", 1) # "Under"(2)+ "stars,"(1)+ "dreams"(1)+ "awake"(2)=6; needs 1.
    ]

    # Choose a random template from each set
    template1, req1 = random.choice(line1_templates)
    template2, req2 = random.choice(line2_templates)
    template3, req3 = random.choice(line3_templates)

    # Choose a label (or fallback) for each placeholder with the required syllable count.
    a1 = choose_label_for(req1, label_pool, fallbacks)
    a2 = choose_label_for(req2, label_pool, fallbacks)
    a3 = choose_label_for(req3, label_pool, fallbacks)

    line1 = template1.format(a=a1)
    line2 = template2.format(a=a2)
    line3 = template3.format(a=a3)

    return f"{line1}\n{line2}\n{line3}"

if __name__ == "__main__":
    image_path = "ithaca.jpg"  # Update with your image path as needed
    labels = analyze_image(image_path)
    haiku = create_template_haiku(labels)
    print("\n🌅 Generated Haiku:")
    print(haiku)


Google Cloud Vision API is installed and ready!
photo-haiku.json
Vision API client successfully initialized!

🎯 Detected Labels:
- City (Confidence: 0.96)
- Residential area (Confidence: 0.95)
- Urban area (Confidence: 0.95)
- Neighbourhood (Confidence: 0.94)
- Landscape (Confidence: 0.92)
- Roof (Confidence: 0.90)
- Bird's-eye view (Confidence: 0.90)
- Human settlement (Confidence: 0.88)
- Urban design (Confidence: 0.88)
- Suburb (Confidence: 0.84)

😊 No faces detected.

🌅 Generated Haiku:
Calm Bird's-eye view sings
In the Landscape light of night
Quiet City dreams
